In [1]:
# 1 · install required libraries
# !pip -q install --upgrade plotly anywidget pandas polars numpy


print("✅ Libraries ready & custom-widget manager enabled")

✅ Libraries ready & custom-widget manager enabled


In [1]:
# 2 · ensure project code is importable and reload edited modules
import sys, pathlib, importlib, pprint

root = pathlib.Path(".").resolve()
if str(root) not in sys.path:
    sys.path.append(str(root))

import helpers.data_loader as dl
import widget.nmf_widget  as nmfw
importlib.reload(dl)
importlib.reload(nmfw)

print("✅ helpers & widget packages reloaded from", root)

✅ helpers & widget packages reloaded from /net/users/voy/nmfwidget


In [2]:
# 3 · quick data sanity-check (optional)
H, ids, cancers = dl.get_prepared_data("config.json")
print("Matrix shape:", H.shape)
print("Sample IDs  :", len(ids))
print("Cancer types:", len(cancers))

Loading data from: comps/all_H_component_contributions.csv
Matrix shape: (410, 16)
Sample IDs  : 410
Cancer types: 410


In [3]:
# 4 · build & display the widget
from widget import create_nmf_widget

w = create_nmf_widget("config.json")
display(w)          # ‹— anywidget wrapper

# fallback that always works (optional):
# w.figure.show()

Loading data from: comps/all_H_component_contributions.csv
Loading component colors from: nmf_component_color_map.json
Loaded component colors: {'Comp_0': '#332288', 'Comp_1': '#88CCEE', 'Comp_2': '#117733', 'Comp_3': '#DDCC77', 'Comp_4': '#CC6677', 'Comp_5': '#882255', 'Comp_6': '#AA4499', 'Comp_7': '#44AA99', 'Comp_8': '#999933', 'Comp_9': '#DDDDDD', 'Comp_10': '#E69F00', 'Comp_11': '#56B4E9', 'Comp_12': '#009E73', 'Comp_13': '#F0E442', 'Comp_14': '#0072B2', 'Comp_15': '#D55E00'}
Component color mapping: {'Component 1': {'color': '#332288', 'matched_key': 'Comp_0'}, 'Component 2': {'color': '#88CCEE', 'matched_key': 'Comp_1'}, 'Component 7': {'color': '#AA4499', 'matched_key': 'Comp_6'}, 'Component 11': {'color': '#E69F00', 'matched_key': 'Comp_10'}, 'Component 3': {'color': '#117733', 'matched_key': 'Comp_2'}, 'Component 5': {'color': '#CC6677', 'matched_key': 'Comp_4'}, 'Component 8': {'color': '#44AA99', 'matched_key': 'Comp_7'}, 'Component 9': {'color': '#999933', 'matched_key': 

In [8]:
import pandas as pd
import umap
import jscatter
import seaborn as sns
from helpers.data_loader import get_prepared_data
import warnings

# For Google Colab, you might need to uncomment the following line:
# jscatter.enable_colab()

# 1. Load data using your custom loader specified in config.json
component_matrix, sample_ids, cancer_types = get_prepared_data("config.json")

# 2. Perform UMAP dimensionality reduction and capture any warnings
umap_warning_message = ""
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")  # Capture all warnings
    u = umap.UMAP(n_components=2, random_state=42).fit_transform(component_matrix)
    if w and issubclass(w[-1].category, UserWarning):
        umap_warning_message = str(w[-1].message)

# 3. If a warning was captured, print it
if umap_warning_message:
    print(f"Warning from UMAP library: {umap_warning_message}")

# 4. Generate color key with the 'turbo' palette
unique_labels = sorted(list(set(cancer_types)))
color_key = dict(zip(unique_labels, sns.color_palette("turbo", len(unique_labels))))

# 5. Create the interactive jscatter plot object
scatter = jscatter.Scatter(
    data=pd.DataFrame({
        "x": u[:, 0],
        "y": u[:, 1],
        "Cancer Type": cancer_types,
        "Sample ID": sample_ids
    }),
    x="x",
    y="y",
    color_by="Cancer Type",
    color_map=color_key,
    height=600
).tooltip(
    properties=["Sample ID", "Cancer Type"]
).size(
    default=5  # Increase the size of all dots
).options({
    'aspectRatio': 1.0  # Set the aspect ratio to 1 for equal axes
})

# 6. Explicitly call .show() to display the plot as a widget
scatter.show()


Loading data from: comps/all_H_component_contributions.csv
Warning from UMAP library: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.


In [4]:
# w.figure.show()

In [7]:
# Check available renderers
import plotly.io as pio
print(pio.renderers)

# Try forcing the notebook renderer
pio.renderers.default = "notebook_connected"
w.figure.show()

# Check if widget frontend is registered
from ipywidgets import Widget
print(Widget.widget_types)

Renderers configuration
-----------------------
    Default renderer: 'notebook'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']



In [8]:
# Save as HTML and view in browser
w.figure.write_html("nmf_heatmap.html")

# Or use static image export
# w.figure.write_image("nmf_heatmap.png", width=1200, height=800)

In [9]:
# Export interactive HTML with sorting controls
from widget.nmf_widget import create_html_with_sorting_controls
html_file = create_html_with_sorting_controls(w, "interactive_nmf.html")
print(f"Interactive HTML with sorting saved to {html_file}")

Interactive HTML with sorting saved to interactive_nmf.html
